In [4]:
import pandas as pd
import streamlit as st
import snowflake.connector
from dotenv import load_dotenv
import os

In [12]:
load_dotenv()
rapidapi_key = os.getenv('RAPIDAPI_KEY')
rapidapi_host = "linkedin-job-search-api.p.rapidapi.com"
snowflake_password = os.getenv('SNOWFLAKE_PASSWORD')

## Snowflake Connection

In [13]:
# Connect to Snowflake
#Establish a connection to Snowflake

def connect_to_snowflake():
    try:

        conn = snowflake.connector.connect(
            user="NIKKILW2025",
            password=snowflake_password,
            account="gbszkwp-by30611",
            warehouse="SNOWFLAKE_LEARNING_WH",
            database="linkedin_db",
            schema="linkedin_raw"
        )
        print("Connection to Snowflake established successfully.")
        return conn
    except Exception as e:
        print(f"Error connecting to Snowflake: {e}")
        return None

conn = connect_to_snowflake()

Connection to Snowflake established successfully.


In [ ]:
def query_job_total(conn):
    query = """
        SELECT JOB_CATEGORY as Title,
        COUNT(DISTINCT ID) as Total_Jobs
        FROM LINKEDIN_JOB_API_CLEANED_DATA
        WHERE
        lower(title) LIKE '%data engineer%'
        or lower(title) LIKE '%data analyst%'
        or lower(title) LIKE '%data scientist%'
        GROUP BY JOB_CATEGORY
        ORDER BY Title ASC
    """
    df_job_total = pd.read_sql(query, conn)
    return df_job_total

df_job_total = query_job_total(conn)
df_job_total


/tmp/ipykernel_15211/449854605.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_job_total = pd.read_sql(query, conn)


,TITLE,TOTAL_JOBS
0,Data Analyst,102
1,Data Engineer,153
2,Data Scientist,51


In [ ]:
# def job_data_summary(df_job_total):
#     """
#     3 Summary Numbers of Total Jobs by Title
#     """
#     col1, col2, col3 = st.columns(3)

#     col1.metric(label='Data Analyst', value=int(df_job_total.iloc[0,1]))
#     col2.metric(label='Data Engineer', value=int(df_job_total.iloc[1,1]))
#     col3.metric(label='Data Scientist', value=int(df_job_total.iloc[2,1]))


np.int64(102)

## Section  - Dashboard Title


In [11]:
#query the min and max dates for dashboard subheading

def job_dates(conn):
    query = """
        SELECT
        MIN(DATE) AS min_date,
        MAX(DATE) AS max_date
        FROM mart_total_jobs_daily
    """

    df_dates = pd.read_sql(query, conn)
    conn.close()
    return df_dates

df_dates = job_dates(conn)
df_dates

/tmp/ipykernel_15211/1653369068.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_dates = pd.read_sql(query, conn)


,MIN_DATE,MAX_DATE
0,2025-04-21,2025-05-06


In [10]:
st.title("Australia Data Job Trend Dashboard")
# st.subheader(f"data date range {df_dates['MIN_DATE']} - {df_dates['MAX_DATE']}")

2025-05-06 06:24:12.156 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-06 06:24:12.157 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()